## Q1

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np

im = cv.imread('the_berry_farms_sunflower_field.jpeg', cv.IMREAD_REDUCED_COLOR_4)
im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
plt.imshow(im)    

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

input_image = cv.cvtColor(im, cv.COLOR_RGB2GRAY)
sigma_values = np.arange(5, 10.5, 0.5)

for sigma in sigma_values:
    image_copy = im.copy()
    scale_space = np.empty((im.shape[0], im.shape[1], 500), dtype=np.float64)
    sigmas = np.arange(sigma, sigma + 0.5, 0.01)
    
    for i, current_sigma in enumerate(sigmas):
        log_hw = 3 * np.ceil(np.max(sigmas))
        X, Y = np.meshgrid(np.arange(-log_hw, log_hw + 1, 1), np.arange(-log_hw, log_hw + 1, 1))
        log_filter = 1 / (2 * np.pi * current_sigma ** 2) * (X ** 2 / (current_sigma ** 2) +\
             Y ** 2 / (current_sigma ** 2) - 2) * np.exp(-(X ** 2 + Y ** 2) / (2 * current_sigma ** 2))
        filtered_log = cv.filter2D(input_image, cv.CV_64F, log_filter)
        scale_space[:, :, i] = filtered_log

    max_indices = np.unravel_index(np.argmax(scale_space, axis=None), scale_space.shape)
    radius = sigmas[max_indices[2]] * np.sqrt(2)
    cv.circle(image_copy, (int(max_indices[1]), int(max_indices[0])), int(radius), (0, 255, 0), 2)

    fig, ax = plt.subplots()
    plt.title('Radius = ' + str(radius) + ', Sigma = ' + str(sigmas[max_indices[2]]))
    plt.imshow(image_copy)





In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

input_gray_image = cv.cvtColor(im, cv.COLOR_RGB2GRAY)
k_values = np.arange(1, 11, 0.125)

for k_value in k_values:
    scale_space = np.empty((im.shape[0], im.shape[1], 125), dtype=np.float64)
    sigmas = np.arange(k_value, k_value + 0.125, 0.001)
    
    for i, current_sigma in enumerate(sigmas):
        log_hw = 3 * np.ceil(np.max(sigmas))
        X, Y = np.meshgrid(np.arange(-log_hw, log_hw + 1, 1), np.arange(-log_hw, log_hw + 1, 1))
        log_filter = 1 / (2 * np.pi * current_sigma ** 2) * (X ** 2 / (current_sigma ** 2) + Y ** 2 / (current_sigma ** 2) - 2) * np.exp(-(X ** 2 + Y ** 2) / (2 * current_sigma ** 2))
        filtered_log = cv.filter2D(input_gray_image, cv.CV_64F, log_filter)
        scale_space[:, :, i] = filtered_log

    max_indices = np.unravel_index(np.argmax(scale_space, axis=None), scale_space.shape)
    radius = sigmas[max_indices[2]] * np.sqrt(2)
    cv.circle(im, (int(max_indices[1]), int(max_indices[0])), int(radius), (0, 255, 0), 2)

fig, ax = plt.subplots()
plt.imshow(im)


## Q2

In [ ]:
from scipy . optimize import minimize
from scipy import linalg
import matplotlib . pyplot as plt
import tikzplotlib
import numpy as np
np.random.seed (0)
N = 100
half_n = N//2
r = 10
x0_gt , y0_gt = 2,3 # Center
s = r /16
t = np.random.uniform(0 , 2*np.pi, half_n )
n = s*np.random.randn( half_n )
x , y = x0_gt + (r + n)*np.cos(t) , y0_gt + (r + n)*np.sin (t)
X_circ = np . hstack ( ( x . reshape ( half_n , 1 ) , y . reshape ( half_n , 1 ) ) )
s = 1.
m, b = -1, 2
x = np.linspace (-12, 12 , half_n )
y = m*x + b + s*np.random.randn ( half_n )
X_line = np.hstack((x.reshape(half_n,1),y.reshape(half_n,1)))
all_points = np.vstack((X_circ,X_line )) # All points
fig , ax = plt . subplots (1 ,1 , figsize =(8 ,8))
ax . scatter ( X_line [ : , 0 ] , X_line [ : , 1 ] , label= ' Line ' )
ax . scatter ( X_circ [ : , 0 ] , X_circ [ : , 1 ] , label= ' Circle ' )
circle_gt = plt. Circle( ( x0_gt , y0_gt ) , r , color= 'g' , fill =False , label= 'Ground truth circle')
ax . add_patch (circle_gt )
ax . plot ( ( x0_gt ) , ( y0_gt ) , '+' , color= 'g' )
x_min , x_max = ax . get_xlim ( )
x_ = np . array ( [ x_min , x_max ] )
y_ = m*x_ + b
plt . plot (x_ , y_ , color= 'm' , label= 'Ground truth line')
plt . legend ( )


In [ ]:
import numpy as np

np.random.seed(0)
num_iterations = 1000
max_distance_threshold = 1.6  
min_sample_count = 2  

def calculate_distance_to_line(point, line_params):
    a, b, d = line_params
    x, y = point
    return np.abs(a * x + b * y - d) / np.sqrt(a**2 + b**2)

def create_line(x_coords, y_coords):
    slope = (y_coords[1] - y_coords[0]) / (x_coords[1] - x_coords[0])
    intercept = y_coords[0] - slope * x_coords[0]
    a = -slope
    b = 1
    d = intercept
    return a, b, d

points = all_points

for _ in range(num_iterations):
    selected_indices = np.random.choice(len(points), size=min_sample_count, replace=False)
    selected_points = points[selected_indices]
    a, b, d = create_line(selected_points[:, 0], selected_points[:, 1])
    norm = np.sqrt(a**2 + b**2)
    a /= norm
    b /= norm
    d /= norm
    
    consensus_set = []
    
    for i, point in enumerate(points):
        if calculate_distance_to_line(point, [a, b, d]) < max_distance_threshold:
            consensus_set.append(i)
    
    if len(consensus_set) > 45:
        break

xc_consensus = points[consensus_set, 0]
yc_consensus = points[consensus_set, 1]
x_coords = points[:, 0]
y_coords = points[:, 1]

plt.figure(figsize=(7, 7))
plt.plot(x_coords, y_coords, 'o', color='b', label='All points')
plt.plot(xc_consensus, yc_consensus, 'o', color='r', label='Consensus points')
plt.legend(loc='best')
plt.show()

In [ ]:
A_matrix = np.hstack((xc_consensus.reshape(len(xc_consensus), 1), np.ones((len(xc_consensus), 1))))
b_vector = yc_consensus.reshape(len(yc_consensus), 1)
x_star = np.linalg.inv(A_matrix.T.dot(A_matrix)).dot(A_matrix.T).dot(b_vector)
m_star = x_star[0]
c_star = x_star[1]

x_range = np.array([np.min(x_coords), np.max(x_coords)])
y_range = m_star * x_range + c_star

plt.figure(figsize=(7, 7))
plt.plot(x_range, y_range, color='m', label='Estimated line')
plt.plot(x_coords, y_coords, 'o', label='Noisy points')
plt.plot(xc_consensus, yc_consensus, '+', color='r', label='Consensus points')
plt.plot(selected_points[:, 0], selected_points[:, 1], 'o', color='g', label='Selected points')
plt.legend(loc='best')



In [ ]:
print(m_star, c_star)

In [ ]:
remaining_points = np.delete(points, consensus_set, axis=0)

# Plot the remaining points
x_remaining = remaining_points[:, 0]
y_remaining = remaining_points[:, 1]

plt.figure(figsize=(7, 7))
plt.plot(x_remaining, y_remaining, 'o', color='b', label='Remaining points')



In [ ]:
import numpy as np

np.random.seed(0)
num_iterations = 1000
max_circle_distance_threshold = 1.1 
min_sample_count = 3  

def calculate_distance_to_circle(point, circle_params):
    x0, y0, r = circle_params
    x, y = point
    return np.abs(np.sqrt((x - x0)**2 + (y - y0)**2) - r)

def fit_circle(x_coords, y_coords):
    x1, x2, x3 = x_coords
    y1, y2, y3 = y_coords
    A_matrix = np.array([[2*(x2-x1), 2*(y2-y1)], [2*(x3-x2), 2*(y3-y2)]])
    b_vector = np.array([x2**2 + y2**2 - x1**2 - y1**2, x3**2 + y3**2 - x2**2 - y2**2])
    x0, y0 = np.linalg.inv(A_matrix).dot(b_vector)
    r = np.sqrt((x1-x0)**2 + (y1-y0)**2)
    return x0, y0, r

new_points = remaining_points
estimated_circle = None

for _ in range(num_iterations):
    selected_indices = np.random.choice(len(new_points), size=min_sample_count, replace=False)
    selected_points = new_points[selected_indices]
    
    x0, y0, r = fit_circle(selected_points[:, 0], selected_points[:, 1])
    
    consensus_set_circle = []
    
    for i, point in enumerate(new_points):
        if calculate_distance_to_circle(point, [x0, y0, r]) < max_circle_distance_threshold:
            consensus_set_circle.append(i)
    
    if len(consensus_set_circle) > 35:
        estimated_circle = [x0, y0, r]
        break

x_coords = new_points[:, 0]
y_coords = new_points[:, 1]
xc_circle = new_points[consensus_set_circle, 0]
yc_circle = new_points[consensus_set_circle, 1]

plt.figure(figsize=(8, 8))
plt.plot(x_coords, y_coords, 'o', label='Noisy points')
plt.plot(xc_circle, yc_circle, '+', color='r', label='Consensus points')
plt.plot(selected_points[:, 0], selected_points[:, 1], 'o', color='r', label='Selected points')
estimated_circle_patch = plt.Circle((estimated_circle[0], estimated_circle[1]), estimated_circle[2], color='m', fill=False, label='Estimated circle')
ax = plt.gca()
ax.add_patch(estimated_circle_patch)
plt.legend(loc='best')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()


In [ ]:
plt.figure(figsize=(12, 12))

plt.plot(x_coords, y_coords, 'o')
plt.plot(xc_consensus, yc_consensus, 'o', color='r', label='Line inliers')
plt.plot(xc_circle, yc_circle, 'o', color='g', label='Circle inliers')

x_temp_range = np.array([x_min, x_max])
y_temp_range = (-1) * x_range + 2
plt.plot(x_temp_range, y_temp_range, color='m', label='Ground truth line')

circle_truth = plt.Circle((x0_gt, y0_gt), r, color='c', fill=False, label='Ground truth circle')
ax = plt.gca()
ax.add_patch(circle_truth)

plt.plot(x_range, y_range, color='y', label='Estimated line')

circle_estimated = plt.Circle((estimated_circle[0], estimated_circle[1]), estimated_circle[2], color='k', fill=False, label='Estimated circle')
ax = plt.gca()
ax.add_patch(circle_estimated)

plt.plot(selected_points_C[:, 0], selected_points_C[:, 1], 'o', color='y', label='Selected points for circle')
plt.plot(selected_points_L[:, 0], selected_points_L[:, 1], 'o', color='c', label='Selected points for line')
plt.legend(loc='best')


## Q3

In [14]:
def measure_images(image_1, image_2):  
    coordinates = [] 
    def click_event(event, x, y, flags, param):
        nonlocal coordinates
        if event == cv.EVENT_LBUTTONDOWN:
            coordinates.append((x, y))
            cv.circle(image_1, (x, y), 5, (0, 0, 255), -1) 
            cv.imshow('Image', image_1) 
            if len(coordinates) == 4:
                cv.destroyAllWindows()
    cv.imshow('Image', image_1)
    cv.setMouseCallback('Image', click_event) 
    while len(coordinates) < 4:
        cv.waitKey(1)
    image_1_points = np.array(coordinates, dtype=np.float32)
    image_2_width = image_2.shape[1] 
    image_2_height = image_2.shape[0] 
    image_2_points = np.array([[0, 0], [image_2_width, 0], [image_2_width, image_2_height], [0, image_2_height]], dtype=np.float32)
    
    return image_1_points, image_2_points

def superimpose_images(image_1, image_2, image_1_points, image_2_points, alpha , beta):
    # Compute the homography matrix
    homography_matrix= cv.findHomography(image_2_points, image_1_points)[0]

    # Warp the flag image to fit the architectural image
    image_2_warped = cv.warpPerspective(image_2, homography_matrix, (image_1.shape[1], image_1.shape[0]))

    #Blend the images
    blended_image = cv.addWeighted(image_1, alpha, image_2_warped, beta, 0)
    
    return blended_image



In [ ]:
import cv2 as cv
import numpy as np

image_1 = cv.imread('building.jpg', cv.IMREAD_COLOR)
image_2 = cv.imread('flag.png')
image_1_points, image_2_points = measure_images(image_1, image_2)
image_1 = cv.imread('building.jpg', cv.IMREAD_COLOR)
blended_image = superimpose_images(image_1, image_2, image_1_points, image_2_points , 0.9 , 0.4)
 
# Display original images and super imposed image
plt.imshow(cv.cvtColor(blended_image, cv.COLOR_BGR2RGB))



In [ ]:
image_1 = cv.imread('burj.jpeg' )
image_2 = cv.imread('leo.jpg')
image_1_points, image_2_points = measure_images(image_1, image_2)
image_1 = cv.imread('burj.jpeg')
blended_image = superimpose_images(image_1, image_2, image_1_points, image_2_points , 0.7 , 0.9)
 
# Display original images and super imposed image
plt.imshow(cv.cvtColor(blended_image, cv.COLOR_BGR2RGB))

In [ ]:
image_1 = cv.imread('rock.jpg' )
image_2 = cv.imread('sl.jpeg')
image_1_points, image_2_points = measure_images(image_1, image_2)
image_1 = cv.imread('rock.jpg')
blended_image = superimpose_images(image_1, image_2, image_1_points, image_2_points , 0.9 , 0.4)
 
# Display original images and super imposed image
plt.imshow(cv.cvtColor(blended_image, cv.COLOR_BGR2RGB))

## Q4

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
im1, im5 = cv.imread("img1.ppm"), cv.imread("img5.ppm")

sift = cv.SIFT_create()
key_points_1, descriptors_1 = sift.detectAndCompute(im1,None) 
key_points_2, descriptors_2 = sift.detectAndCompute(im5,None)
bf_match = cv.BFMatcher(cv.NORM_L1, crossCheck=True) 
matches = sorted(bf_match.match(descriptors_1, descriptors_2), key = lambda x:x.distance)

im = cv.drawMatches(im1, key_points_1, im5, key_points_2, matches[:250], im5, flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS) 
fig, ax = plt.subplots(figsize=(12,12))
im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
ax.set_title("SIFT Features"), ax.imshow(im)
plt.show()

In [ ]:
import numpy as np
from random import randint
img2, img3, img4, img1, img5 = cv.imread("img2.ppm"), cv.imread("img3.ppm"), cv.imread("img4.ppm"), cv.imread("img1.ppm"), cv.imread("img5.ppm")
im1, im5, im2, im3, im4 = cv.cvtColor(img1, cv.COLOR_BGR2GRAY), cv.cvtColor(img5, cv.COLOR_BGR2GRAY), cv.cvtColor(img2, cv.COLOR_BGR2GRAY), cv.cvtColor(img3, cv.COLOR_BGR2GRAY), cv.cvtColor(img4, cv.COLOR_BGR2GRAY)

ims = [im1, im2, im3, im4, im5]

def random_number(n, t):
    l = np.random.randint(n, size=t)
    m = np.zeros(np.shape(l))
    
    for i in range(len(l)):
        m[i] = np.sum(l==l[i])
    if np.sum(m) == len(m):
        return l
    else:
        return random_number(n,t)

def Homography(p1, p2):
    x1, y1, x2, y2, x3, y3, x4, y4 = p2[0], p2[1], p2[2], p2[3], p2[4], p2[5], p2[6], p2[7] 
    x1T, x2T, x3T, x4T = p1[0], p1[1], p1[2], p1[3]
    zero_matrix = np.array([[0], [0], [0]])

    matrix_A = np.concatenate((np.concatenate((zero_matrix.T,x1T, -y1*x1T), axis = 1), np.concatenate((x1T, zero_matrix.T, -x1*x1T), axis = 1),
                            np.concatenate((zero_matrix.T,x2T, -y2*x2T), axis = 1), np.concatenate((x2T, zero_matrix.T, -x2*x2T), axis = 1),
                            np.concatenate((zero_matrix.T,x3T, -y3*x3T), axis = 1), np.concatenate((x3T, zero_matrix.T, -x3*x3T), axis = 1),
                            np.concatenate((zero_matrix.T,x4T, -y4*x4T), axis = 1), np.concatenate((x4T, zero_matrix.T, -x4*x4T), axis = 1)), axis = 0, dtype=np.float64)
    W, v = np.linalg.eig(((matrix_A.T)@matrix_A))
    temph= v[:,np.argmin(W)]
    H = temph.reshape((3,3))
    return H

p, s, e = 0.99, 4, 0.5
N = int(np.ceil(np.log(1-p)/np.log(1-((1-e)**s))))
Hs = []
for i in range(4):
    sift = cv.SIFT_create()
    key_points_1, descriptors_1 = sift.detectAndCompute(ims[i],None) #sifting
    key_points_2, descriptors_2 = sift.detectAndCompute(ims[i+1],None)
    bf_match = cv.BFMatcher(cv.NORM_L1, crossCheck=True)  #feature matching
    matches = sorted(bf_match.match(descriptors_1, descriptors_2), key = lambda x:x.distance)

    Source_Points = [key_points_1[k.queryIdx].pt for k in matches]
    Destination_Points = [key_points_2[k.trainIdx].pt for k in matches]
    threshold, best_inliers, best_H = 2, 0, 0

    for i in range(N):
        ran_points = random_number(len(Source_Points)-1, 4)
        f_points = []
        for j in range(4):
            f_points.append(np.array([[Source_Points[ran_points[j]][0], Source_Points[ran_points[j]][1], 1]]))
        t_points = []
        for j in range(4):
            t_points.append(Destination_Points[ran_points[j]][0]) 
            t_points.append(Destination_Points[ran_points[j]][1])
        H = Homography(f_points,t_points)       
        inliers = 0 
        for k in range(len(Source_Points)):
            X = [Source_Points[k][0], Source_Points[k][1], 1]
            HX = H @ X
            HX /= HX[-1]
            err = np.sqrt(np.power(HX[0]-Destination_Points[k][0], 2) + np.power(HX[1]-Destination_Points[k][1], 2))
            if err < threshold:
                inliers +=1
        if inliers > best_inliers:
            best_inliers = inliers
            best_H = H 
    Hs.append(best_H)
H1_H5 = Hs[3] @ Hs[2] @ Hs[1] @ Hs[0]
H1_H5 /= H1_H5[-1][-1]


print("Computed Homography \n", H1_H5)

transformed_im = cv.warpPerspective(img1, H1_H5, (np.shape(img5)[1], np.shape(img5)[0]))
final = cv.add(img5, transformed_im)

fig, ax = plt.subplots(1,3,figsize=(12,12))
ax[0].imshow(cv.cvtColor(img1,cv.COLOR_BGR2RGB)), ax[0].set_title("Image 1")
ax[1].imshow(cv.cvtColor(img5,cv.COLOR_BGR2RGB)), ax[1].set_title("Image 5")
#ax[2].imshow(cv.cvtColor(transformed_im,cv.COLOR_BGR2RGB)), ax[2].set_title("Transformed Image")
ax[2].imshow(cv.cvtColor(final,cv.COLOR_BGR2RGB)), ax[2].set_title("Stitched Image")
plt.show()